In [1]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 2.7 MB/s eta 0:00:00a 0:00:01


In [2]:
import pandas as pd
import bs4
import requests
import openpyxl
import json
from tqdm.auto import tqdm

## Web Scraping - BeautifulSoup
web scraping news from **thestandard**

In [3]:
i_page = 1
title_list = []
date_list = []
detail_list = []

while i_page <= 2:
  data = requests.get(f'https://thestandard.co/page/{i_page}/?s=%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%AA%E0%B8%9A%E0%B8%84%E0%B8%A7%E0%B8%B2%E0%B8%A1%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%A3%E0%B9%87%E0%B8%88%E0%B9%83%E0%B8%99%E0%B8%AD%E0%B8%B2%E0%B8%8A%E0%B8%B5%E0%B8%9E&search=')
  soup = bs4.BeautifulSoup(data.text)
    
  for c in soup.find_all('div', class_='col-md-8'):
    title_list.append(c.find('h3', class_= 'news-title').find('a').text.strip())
    date_list.append(c.find('div', class_='date').text.strip())
    detail_list.append(c.find('div', class_='desc').text.strip())
  print('Complete page number: ',i_page)
  i_page += 1

Complete page number:  1
Complete page number:  2


In [38]:
data = {'title': title_list,
       'date': date_list ,
       'detail': detail_list}

table = pd.DataFrame(data)
#table = table.set_index('title')

## Prepare Data

In [39]:
table.columns
#check data type
table.dtypes
#check missing values
table.isna().sum().sort_values(ascending = False)

title     0
date      0
detail    0
dtype: int64

In [40]:
table['date_split'] = table['date'].str.split('\s')

In [41]:
table['D'] = table['date_split'].str[0]
table['M'] = table['date_split'].str[1]
table['Y']  = table['date_split'].str[2]

In [42]:
table['M'].unique()

array(['ตุลาคม', 'สิงหาคม', 'พฤษภาคม', 'มีนาคม', 'กุมภาพันธ์', 'มกราคม',
       'ธันวาคม'], dtype=object)

In [43]:
table['M'].replace({
    'มกราคม':'01',
    'กุมภาพันธ์':'02',
    'มีนาคม' :'03',
    'เมษายน':'04',
    'พฤษภาคม':'05',
    'มิถุนายน':'06',
    'กรกฎาคม':'07',
    'สิงหาคม':'08',
    'กันยายน':'09',
    'ตุลาคม':'10',
    'พฤศจิกายน':'11',
    'ธันวาคม': '12'
    }, inplace=True)

In [44]:
table['DATE'] = table['D']+'-'+table['M']+'-'+table['Y']

In [45]:
#table['DATE'] = pd.to_datetime(table['DATE'], errors='ignore')

In [46]:
table.drop(['date', 'date_split', 'D','M','Y'], axis=1, inplace=True)

In [47]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   17 non-null     object
 1   detail  17 non-null     object
 2   DATE    17 non-null     object
dtypes: object(3)
memory usage: 536.0+ bytes


In [48]:
table = table.to_dict('records')

In [49]:
table

[{'title': 'นายกฯ ขอให้ทุกคนรักชาติ มีศรัทธาในการทำงาน เกษตรกรก็ประสบความสำเร็จในอาชีพได้',
  'detail': 'วันนี้ (3 ตุลาคม) \u202aเวลา 13.30 น. ณ โรงแรมมิราเคิล แกรนด์ กรุงเทพฯ พล.อ. ประยุทธ์ จันทร์โอชา นายกรัฐมนตรี และรัฐมนตรีว่าการกระทรวงกลาโหม เป็นประธานมอบนโยบายแก่สภาเกษตรกรแห่งชาติ ในโอกาสจัดประชุมสภาเกษตรกรแห่งชาติ ปีงบประมาณ 2563 พร้อมด้วย สมคิด จาตุศรีพิทักษ์ รองนายกรัฐมนตรี เฉลิมชัย ศรีอ่อน รัฐมนตรีว่าการกระทรวงเกษตรและสหกรณ์ สมาชิกสภาเกษตรกรแห่งชา...',
  'DATE': '3-10-2019'},
 {'title': '7 Things We Love About Hailey Bieber ผู้หญิงที่เป็นมากกว่าแค่ภรรยาศิลปินชื่อดัง',
  'detail': 'ใครที่ติดตามข่าวสารและป๊อปคัลเจอร์อยู่เสมอจะต้องรู้จักชื่อของ Hailey Bieber (เฮลีย์ บีเบอร์) นางแบบ นักธุริจ และภรรยาสาวของศิลปินแห่งยุคอย่าง Justin Bieber (จัสติน บีเบอร์) แม้ภายนอกเธอจะดูเก๋โก้และเป็นแฟชั่นไอคอนให้กับเจนใหม่ แต่จริงๆ แล้วที่หลายคนหลงเสน่ห์เธอกลับเป็นเพราะบุคลิคที่เข้าถึงได้และเป็นกันเองของเธอต่างหาก\n\xa0\nนอกจากปกนิตยสารแล...',
  'DATE': '23-10-2022'},
 {'title': 'POP TIP: เคล็ดลั

In [51]:
with open('news1.json', 'w') as f:json.dump(table, f)

In [52]:
#df = pd.DataFrame(table)
#df['DATE'] = df['DATE'].dt.strftime('%Y-%m-%d %H:%M:%S')
#print (df.dtypes)

In [53]:
#table = df.to_dict('records')
#table

In [54]:
#with open('news.json', 'w') as f:json.dump(table, f)

# Upload data to DataPlatform

In [55]:
from dsmlibrary.datanode import DataNode

In [56]:
dir_raw_id = 54

In [57]:
datanode = DataNode()

Init DataNode sucessful!


In [58]:
datanode.upload_file(directory_id=dir_raw_id, file_path='news1.json')

  0%|          | 0.00/39.7k [00:00<?, ?B/s]

{'id': 299,
 'msg': 'file news1.json create sucessful',
 'path': 'natthakan/DSI310-05/rawData/news1.json',
 'uploaded': True}